In [2]:
import os
import requests
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
import datetime as dt
import yfinance as yf
from bs4 import BeautifulSoup

path = Path("data/raw")
path.mkdir(parents=True,exist_ok=True)

load_dotenv()
try:
    api_key = os.getenv("API_KEY")
except Exception as e:
    print("Error occur",e)

In [48]:
#useable function
def validate(df:pd.DataFrame,required_columns):
    missing = [c for c in required_columns if c not in df.columns ]
    return {"missing":missing,"shape":df.shape,"none_count":df.isna().sum().sum}
def time_stamp():
    return dt.datetime.now().strftime("%Y%m%d-%H%M%S")
def save_csv(df:pd.DataFrame,prefix:str,**meta):
    mid = '_'.join([f"{k}-{v}" for k,v in meta.items()])
    save_path = path / f"{prefix}_{mid}_{time_stamp()}.csv"
    df.to_csv(save_path,index=False)
    print(f"saved path:{save_path}")
    return save_path

In [5]:
#request stock data from public API alpha vantage
symbol = "NVDA"
if api_key:
    url = "https://www.alphavantage.co/query"
    params = {
        "function":"TIME_SERIES_DAILY",
        "symbol":symbol,
        "outsize":"compact",
        "data_type":"json",
        "apikey":api_key
    }
    r = requests.get(url=url,params=params)
    js = r.json()
    js.keys()
    series = js["Time Series (Daily)"]
    df_api = pd.DataFrame(series).T.rename_axis("date").reset_index()
    df_api["date"] = pd.to_datetime(df_api["date"])
    df_api.rename(columns={"1. open":"open","2. high":"high","3. low":"low","4. close":"close","5. volume":"volume"},inplace=True)
    for column in df_api.columns[1:]:
        df_api[column] = pd.to_numeric(df_api[column])
else:
    df_api = yf.download(tickers=symbol,period="6mo",interval="1d",auto_adjust=False)
    df_api = df_api.reset_index()
    df_api.rename(columns={"Open":"open","High":"high","Low":"low","Close":"close","Volume":"volume"},inplace=True)
df_api["date"].sort_values().reset_index(drop=True)
# df_api

0    2025-03-27
1    2025-03-28
2    2025-03-31
3    2025-04-01
4    2025-04-02
        ...    
95   2025-08-13
96   2025-08-14
97   2025-08-15
98   2025-08-18
99   2025-08-19
Name: date, Length: 100, dtype: datetime64[ns]

In [ ]:
#validate dataframe for required column 
required = ["open","high","low","close","volume","adj close"]
validate(df_api,required_columns=required)

{'missing': ['adj close'],
 'shape': (100, 6),
 'none_count': <bound method Series.sum of date      0
 open      0
 high      0
 low       0
 close     0
 volume    0
 dtype: int64>}

In [ ]:
#save csv file to path
_ = save_csv(df_api, prefix='api', source='alpha' if api_key else 'yfinance', symbol=symbol)

saved path:data/raw/api_source-alpha_symbol-NVDA_20250817-212610.csv


In [73]:
#Scrape a Small Table
table_url = "https://home.treasury.gov/resource-center/data-chart-center/interest-rates/TextView?type=daily_treasury_yield_curve&field_tdr_date_value=2025"
headers = {"User-Agent":"AFE-Homework"}
r = requests.get(url=table_url,headers=headers)
soup = BeautifulSoup(r.text,"html.parser")
table = soup.find_all("table")
# Select the first table from the ResultSet
main_table = table[0]
rows = [[c.get_text(strip=True) for c in tr.find_all(["th","td"])] for tr in main_table.find_all("tr") if tr.find_all(["th","td"])]
header, *data = [r for r in rows if r]
df_scrape = pd.DataFrame(data, columns=header)

In [85]:
required = {"1 Yr","2 Yr","3 Yr","5 Yr"}
validate(df_scrape,required_columns=required)
_ = save_csv(df_scrape,prefix="scrape",source="US_Department_of_the_treasury",table="Daily_Treasury_Par_Yield_Curve_Rates_2025")

saved path:data/raw/scrape_source-US_Department_of_the_treasury_table-Daily_Treasury_Par_Yield_Curve_Rates_2025_20250817-231232.csv


## Notes&Sources ##
**API Sources**: Alpha vantage or yfinance as fallback<br>
**Scrape Source**: US department of the treasury<br>
**Secrets**: `.env` with `ALPHAVANTAGE_API_KEY`; do not commit `.env`<br>
**params&validation**:fetch daily stock data(OHLCV,adjust close)from alpha vantage or yfinance as fallback;fetch US treasury yield curve rates for 1yr,2yr,3yr,5yr<br>
**assumptions&risks**:assume all the data are accurate and reliable;missing data might cause deviation for future analysis